In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
from bs4 import BeautifulSoup

# WEBSCRAPING STEP
all_data = ""
for page in range(2019, 2023):
    season = int(page + 1)
    url = "https://www.hockey-reference.com/leagues/NHL_{page}_skaters.html".format(page =page+1)

    print(url)

    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    
    for records in soup.select('tr[class != "colhead"]'):
        player_data = ""
        for data in records.findAll('td'):
            player_data = player_data + "," + data.text
        all_data = all_data + '\n' + player_data[1:]
    all_data = all_data + '\n' + "SEASON: " + str(season - 1) + "-" + str(season)
stat_headers = "Player,Age,Team,Pos,GP,G,A,PTS,+/-,PIM,PS,EVG,PPG,SHG,GW,EVA,PPA,SHA,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%"
file = open(os.path.expanduser('nhl_player_stat_databasetest.csv'), 'wb')
all_data = stat_headers + '\n' + all_data
file.write(bytes(all_data, encoding='ascii', errors='ignore'))


# PANDAS & NUMPY TO FILTER/MODIFY DATA
df = pd.read_csv('nhl_player_stat_databasetest.csv')
columns = stat_headers

all_players_teams = df[['Player','Team']]   # all data in players & teams cols
all_tot = (df[df['Team'] == 'TOT'])   # all data for TOT players
subset_tot = all_tot[['Player','Team']]   # player & team name data for TOT players

# creating list containing names of TOT players:
tot_players = []
for num in (subset_tot).index.values:
  player_name = subset_tot._get_value(num,'Player')
  tot_players.append(player_name)

# remove next appearances of TOT player in list:
to_remove = []
for player in tot_players:
  lst = list(np.where(df['Player'] == player)[0])
  for num in lst:
    num += 1
    next_name = all_players_teams._get_value(num,'Player')
    if 'SEASON' in next_name:
      break
    elif next_name == player:
      (to_remove).append(num)

players_names = (df['Player']).values.tolist()  # list of players names

seasons = []
for entry in players_names:
  if 'SEASON' in entry:
    seasons.append(df[df['Player'] == entry].index.values)
# identify where last season begins
val = len(seasons) - 2
index_last_season = (seasons.pop(val))[0]

# check players from all but last season
past_players = []
for num in range(index_last_season):
  past_players.append(all_players_teams._get_value(num,'Player'))

# create list of names of players in current season
curr_players = []
for num in range(index_last_season+1,seasons.pop()[0]):
  curr_players.append(all_players_teams._get_value(num,'Player'))

for player in past_players:
  if player not in curr_players:
    print(f'player: {player}')
    # remove all instances of player by appending index to to_remove
    indices_to_remove = list(np.where(df['Player'] == player)[0])
    print(f'indices_to_remove: {indices_to_remove}')

  for num in indices_to_remove:
    to_remove.append(num)

# remove data for TOT players + inactive players in to_remove
df.drop(to_remove, axis=0, inplace=True)

# check for num of games played
df.replace(np.nan,0)
df = df[df['GP'] > 9]
df['PLAYER_RATING'] = df["EVG"] * 0.8 + df["PPG"] * 0.65 + df["SHG"] * 0.8 + df["GW"] * 0.1 + df["EVA"] * 0.55 + df["PPA"] * 0.45 + df["SHA"] * 0.55 + df["S"] * 0.075 + df["BLK"] * 0.05 + df["PIM"] * 0.05 + df["FOW"] * 0.025 - df["FOL"] * 0.025 + df["HIT"] * 0.025 + df["+/-"] * 0.15
df['PR/GP'] = df["PLAYER_RATING"] / df["GP"] * 10

# pr_only_df = asdasdasd (this dataframe only has player rating and player rating per games played)
pd.set_option('display.max_rows', None)
# print(df.to_string)
# df.describe()
df.describe()
# df.plot(x=["G"], y=["GP"])

https://www.hockey-reference.com/leagues/NHL_2020_skaters.html
https://www.hockey-reference.com/leagues/NHL_2021_skaters.html
https://www.hockey-reference.com/leagues/NHL_2022_skaters.html
https://www.hockey-reference.com/leagues/NHL_2023_skaters.html
player: Justin Abdelkader
indices_to_remove: [0]
player: Pontus Aberg
indices_to_remove: [1]
player: Vitaly Abramov
indices_to_remove: [2, 1017]
player: Andrew Agozzino
indices_to_remove: [4, 5, 6, 1021, 2051]
player: Andrew Agozzino
indices_to_remove: [4, 5, 6, 1021, 2051]
player: Andrew Agozzino
indices_to_remove: [4, 5, 6, 1021, 2051]
player: Karl Alzner
indices_to_remove: [8]
player: Andy Andreoff
indices_to_remove: [16, 1038, 2066]
player: Anthony Angello
indices_to_remove: [17, 1039, 2067]
player: Artem Anisimov
indices_to_remove: [18, 1040]
player: Cam Atkinson
indices_to_remove: [28, 1048, 2079]
player: David Backes
indices_to_remove: [30, 31, 32, 1050]
player: David Backes
indices_to_remove: [30, 31, 32, 1050]
player: David Backe

,Age,GP,G,A,PTS,+/-,PIM,PS,EVG,PPG,...,S,S%,TOI,BLK,HIT,FOW,FOL,FO%,PLAYER_RATING,PR/GP
count,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,...,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,2515.000000,1608.000000,2515.000000,2515.000000
mean,26.497416,48.936382,8.823857,14.795626,23.619483,0.095030,23.278728,2.701590,6.767793,1.817097,...,88.696620,8.862584,829.754274,37.913718,62.077137,80.615507,80.496223,42.200560,25.991233,4.984511
std,3.942140,18.721675,8.515533,12.286182,19.280235,11.413615,18.691202,2.489423,6.115580,2.960753,...,57.151216,5.235032,408.747940,31.463382,46.550402,159.487274,147.350038,17.070211,17.333165,2.322100
min,18.000000,10.000000,0.000000,0.000000,0.000000,-46.000000,0.000000,-1.000000,0.000000,0.000000,...,1.000000,0.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275000,0.250000
25%,23.000000,38.000000,3.000000,6.000000,9.000000,-7.000000,10.000000,0.800000,2.000000,0.000000,...,44.500000,5.000000,523.000000,16.000000,28.000000,0.000000,0.000000,35.250000,13.087500,3.319444
50%,26.000000,47.000000,6.000000,12.000000,19.000000,-1.000000,18.000000,2.100000,5.000000,0.000000,...,79.000000,8.700000,821.000000,27.000000,51.000000,3.000000,6.000000,45.600000,22.925000,4.516667
75%,29.000000,65.000000,13.000000,21.000000,33.000000,6.000000,32.000000,4.100000,10.000000,3.000000,...,122.000000,12.500000,1114.000000,51.000000,84.000000,62.500000,75.500000,51.100000,35.350000,6.224107
max,39.000000,82.000000,60.000000,85.000000,123.000000,64.000000,151.000000,15.300000,44.000000,26.000000,...,348.000000,29.400000,2144.000000,179.000000,355.000000,991.000000,774.000000,100.000000,109.775000,15.120536
